# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4bb8191b56-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [43]:
# return the properties for Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q13444093 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('p', 'http://www.wikidata.org/prop/direct/P2044'), ('name', 'elevation above sea level')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2326'), ('name', 'GNS Unique Feature ID')]
[('p', 'http://www.wi

In [23]:
# return the properties of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?obj ?objectName
WHERE {
   wd:Q13444093 ?p ?obj .
   # get the label
   ?obj sc:name ?objectName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q68074438'), ('name', 'WikiProject Livius')]
[('obj', 'http://www.wikidata.org/entity/Q189371'), ('name', 'Sneferu')]
[('obj', 'http://www.wikidata.org/entity/Q1066424'), ('name', 'Meidum')]
[('obj', 'http://www.wikidata.org/entity/Q220240'), ('name', 'Huni')]
[('obj', 'http://www.wikidata.org/entity/Q23757'), ('name', 'limestone')]
[('obj', 'http://www.wikidata.org/entity/Q79'), ('name', 'Egypt')]
[('obj', 'http://www.wikidata.org/entity/Q321053'), ('name', 'cenotaph')]
[('obj', 'http://www.wikidata.org/entity/Q1456099'), ('name', 'step pyramid')]
[('obj', 'http://www.wikidata.org/entity/Q30683'), ('name', 'Beni Suef Governorate')]
[('obj', 'http://www.wikidata.org/entity/Q75129112'), ('name', 'Category:Interior of Meidum Pyramid')]
10


In [44]:
# return the instance (P31) of Meidum Pyramid
queryString = """
SELECT DISTINCT ?obj ?instanceName
WHERE {
   wd:Q13444093 wdt:P31 ?obj .
   ?obj sc:name ?instanceName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1456099'), ('instanceName', 'step pyramid')]
1


In [45]:
# return the location (P276) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?location ?locationName
WHERE {
   wd:Q13444093 wdt:P276 ?location .
   ?location sc:name ?locationName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('location', 'http://www.wikidata.org/entity/Q1066424'), ('locationName', 'Meidum')]
1


In [57]:
# return the subproperty (P1647) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?subproperty ?subpropertynName
WHERE {
   wd:Q13444093 wdt:P1647 ?subproperty .
   ?subproperty sc:name ?subpropertynName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [58]:
# return the occupation (P106) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?occupation ?occupationName
WHERE {
   wd:Q13444093 wdt:P106 ?occupation .
   ?occupation sc:name ?occupationName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [59]:
# return the nationality (P27) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?nationality ?nationalityName
WHERE {
   wd:Q13444093 wdt:P27 ?nationality .
   ?nationality sc:name ?nationalityName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [31]:
# return the subclass (P279) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?subclass ?subclassName
WHERE {
   wd:Q13444093 wdt:P279 ?subclass .
   ?subclass sc:name ?subclassName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [61]:
# return the country (P17) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   wd:Q13444093 wdt:P17 ?country .
   ?country sc:name ?countryName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
1


In [62]:
# return the GeoNames ID (P1566) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?GeoNames ?GeoNamesID
WHERE {
   wd:Q13444093 wdt:P1566 ?GeoNames .
   ?GeoNames sc:name ?GeoNamesID .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [63]:
# return the named after (P138) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?namedafter ?namedafterName
WHERE {
   wd:Q13444093 wdt:P138 ?namedafter .
   ?namedafter sc:name ?namedafterName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('namedafter', 'http://www.wikidata.org/entity/Q1066424'), ('namedafterName', 'Meidum')]
1


In [64]:
# return the named after (P7561) of Meidum Pyramid (wd:Q13444093)
queryString = """
SELECT DISTINCT ?category ?categoryName
WHERE {
   wd:Q13444093 wdt:P7561 ?category .
   ?category sc:name ?categoryName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('category', 'http://www.wikidata.org/entity/Q75129112'), ('categoryName', 'Category:Interior of Meidum Pyramid')]
1


In [2]:

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   wd:Q11768 ?p ?obj .
    ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28171280'), ('objName', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('obj', 'http://www.wikidata.org/entity/Q766876'), ('objName', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q714601'), ('objName', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q1138524'), ('objName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent'), ('obj', 'http://www.wikidata.org/entity/Q15'), ('objName', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.or

In [5]:
# Q766876 history of ancient Egypt
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   wd:Q766876 ?p ?obj .
    ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q41493'), ('objName', 'ancient history')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1546344'), ('objName', 'History of Ptolemaic Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('objName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('objName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17524420'), ('objName', 'aspect of history')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q191324'), ('objName', 'Middle Kingdom of Egypt')]
[('p', 'http://ww

Final query for this task

In [7]:
#Q11768 Ancient Egypt
#P2348 time period
# Q766876 history of ancient Egypt
queryString = """
SELECT DISTINCT ?ancientegypt ?ancientegyptName
WHERE {
   wd:Q766876 wdt:P2348 ?ancientegypt .
   ?ancientegypt sc:name ?ancientegyptName .
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('ancientegypt', 'http://www.wikidata.org/entity/Q11768'), ('ancientegyptName', 'Ancient Egypt')]
1


## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [69]:
# write your queries
#obj Q1066424 Meidum

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1066424 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1936'), ('name', 'Digital Atlas of the Roman Empire ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1958'), ('name', 'Trismegistos Geo ID')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2596

In [70]:
#obj Q189371 Sneferu
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q189371 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1309'), ('name', 'EGAXA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1711'), ('name', 'British Museum person or institution ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'se

In [2]:
#Q189371 Sneferu
#P2348 time period
queryString = """
SELECT DISTINCT ?timeperiod ?timeperiodName
WHERE {
   wd:Q189371 wdt:P2348 ?timeperiod .
   ?timeperiod sc:name ?timeperiodName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('timeperiod', 'http://www.wikidata.org/entity/Q177819'), ('timeperiodName', 'Old Kingdom of Egypt')]
[('timeperiod', 'http://www.wikidata.org/entity/Q11768'), ('timeperiodName', 'Ancient Egypt')]
2


In [4]:
#Q189371 Sneferu
#P2348 time period
queryString = """
SELECT DISTINCT ?sName ?pName
WHERE {
   ?s ?p wd:Q177819 .
   ?s ?p wd:Q11768 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Nitocris'), ('pName', 'time period')]
[('sName', 'Sneferu'), ('pName', 'time period')]
[('sName', 'Djedefre'), ('pName', 'time period')]
[('sName', 'Sanakht'), ('pName', 'time period')]
[('sName', 'Khaba'), ('pName', 'time period')]
[('sName', 'Shepseskaf'), ('pName', 'time period')]
[('sName', 'Duaenre'), ('pName', 'time period')]
[('sName', 'Nyuserre Ini'), ('pName', 'time period')]
[('sName', 'Sekhemkhet'), ('pName', 'time period')]
[('sName', 'Unas'), ('pName', 'time period')]
10


In [141]:
#obj Q189371 Sneferu
#p P106 occupation
queryString = """
SELECT DISTINCT ?occupation ?occupationName
WHERE {
   wd:Q189371 wdt:P106 ?occupation .
   ?occupation sc:name ?occupationName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q372436'), ('occupationName', 'statesperson')]
1


In [86]:
#obj Q189371 Sneferu
#p P106 occupation
queryString = """
SELECT DISTINCT ?p ?predicate ?obj ?objName 
WHERE {
   wd:Q189371 ?p ?obj .
   ?p sc:name ?predicate .
   ?obj sc:name ?objName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('predicate', 'child'), ('obj', 'http://www.wikidata.org/entity/Q122426'), ('objName', 'Henutsen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('predicate', 'child'), ('obj', 'http://www.wikidata.org/entity/Q161904'), ('objName', 'Khufu')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('predicate', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q177819'), ('objName', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P53'), ('predicate', 'family'), ('obj', 'http://www.wikidata.org/entity/Q211001'), ('objName', 'Fourth Dynasty of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('predicate', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q256515'), ('objName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('predicate', 'sibling'), ('obj', 'http://www.wikidata.org/entity/Q256515'), ('objName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/dir

In [87]:
#p P97 noble title
#obj Q37110 pharaoh
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    ?s wdt:P97 wd:Q37110 .
   ?s sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('name', 'Djet')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('name', 'Djer')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('name', 'Senusret III')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('name', 'Nitocris')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('name', 'Djedkhonsuefankh')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('name', 'Pepi I Meryre')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('name', 'Merenre Nemtyemsaf II')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('name', 'Smendes')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('name', 'Psusennes I')]
[('s', 'http://www.wikidata.org/entity/Q428633'), ('name', 'Piankh')]
[('s', 'http://www.wikidata.org/entity/Q434877'), ('name', 'Amenemnisu')]
[('s', 'http://www.wikidata.org/entity/Q440496'), ('name', 'Khyan')]
[('s', 'http://www.wikidata.org/entity/Q6198'), ('name', 'Pepi II')]
[('s', 'http://www.wikidata.org/entity/

In [93]:
#p P106 occupation
#p P97 noble title
#obj Q37110 pharaoh
#p Q199861 Khafra
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q199861 ?p ?obj .
   ?p sc:name ?pName.    
   ?obj sc:name ?objName.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('obj', 'http://www.wikidata.org/entity/Q122426'), ('objName', 'Henutsen')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('obj', 'http://www.wikidata.org/entity/Q161904'), ('objName', 'Khufu')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q177819'), ('objName', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P53'), ('pName', 'family'), ('obj', 'http://www.wikidata.org/entity/Q211001'), ('objName', 'Fourth Dynasty of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q456487'), ('objName', 'Khamerernebty I')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('obj', 'http://www.wikidata.org/entity/Q542742'), ('objName', 'Djedefhor')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 

In [133]:
#P106 occupation
#P97 noble title
#Q37110 pharaoh
#Q199861 Khafra
#Q336395 Smendes
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q336395 ?p ?obj .
   ?p sc:name ?pName.    
   ?obj sc:name ?objName.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('obj', 'http://www.wikidata.org/entity/Q1216110'), ('objName', 'Nodjmet')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('obj', 'http://www.wikidata.org/entity/Q434877'), ('objName', 'Amenemnisu')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('objName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('pName', 'position held'), ('obj', 'http://www.wikidata.org/entity/Q37110'), ('objName', 'pharaoh')]
[('p', 'http://www.wikidata.org/prop/direct/P97'), ('pName', 'noble title'), ('obj', 'http://www.wikidata.org/entity/Q37110'), ('objName', 'pharaoh')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q372436'), ('objName', 'statesperson')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'inst

In [167]:
#children number of pharaohs > 8
# Q11768 Ancient Egypt


queryString = """
SELECT ?pharaohName  (COUNT(?childName) AS ?numofchild)
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        wdt:P2348 wd:Q11768 ;
        wdt:P40 ?childName .

}
GROUP BY ?pharaohName 
HAVING (COUNT(?childName) > 8)
ORDER BY DESC (?numofchild)

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohName', 'Sneferu'), ('numofchild', '13')]
[('pharaohName', 'Darius I of Persia'), ('numofchild', '9')]
[('pharaohName', 'Seqenenre Tao'), ('numofchild', '9')]
[('pharaohName', 'Ptolemy I Soter'), ('numofchild', '9')]
4


In [175]:
#Q11768 Ancient Egypt

queryString = """
SELECT DISTINCT ?pName ?objName
WHERE {
    wd:Q11768 ?p ?obj .
    ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'instance of'), ('objName', 'ancient civilization')]
[('pName', 'history of topic'), ('objName', 'history of ancient Egypt')]
[('pName', 'follows'), ('objName', 'Predynastic Period of Egypt')]
[('pName', 'described by source'), ('objName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('pName', 'continent'), ('objName', 'Africa')]
[('pName', 'described by source'), ('objName', 'Great Soviet Encyclopedia (1969–1978)')]
[('pName', 'historical region'), ('objName', 'ancient Near East')]
[('pName', 'instance of'), ('objName', 'historical country')]
[('pName', 'instance of'), ('objName', 'cultural region')]
[('pName', 'described by source'), ('objName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('pName', 'on focus list of Wikimedia project'), ('objName', 'Wikipedia:Vital articles')]
[('pName', 'instance of'), ('objName', 'historical region')]
[('pName', 'follows'), ('objName', 'Prehistoric Egypt')]
[('pName', 'studied by'), ('objName', 'ancient 

In [177]:
#Q11768 Ancient Egypt

queryString = """
SELECT DISTINCT ?sName ?pName
WHERE {
    ?s ?p wd:Q11768 .
    ?s sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'mumia'), ('pName', 'source of material')]
[('sName', 'nomarch'), ('pName', 'applies to jurisdiction')]
[('sName', 'pharaoh'), ('pName', 'applies to jurisdiction')]
[('sName', 'Dioiketes'), ('pName', 'applies to jurisdiction')]
[('sName', 'Armenian Egyptology Centre'), ('pName', 'field of work')]
[('sName', 'Eugene Cruz-Uribe'), ('pName', 'field of work')]
[('sName', 'Gratien-Michel Ollivier-Beauregard'), ('pName', 'field of work')]
[('sName', 'Nikephoros'), ('pName', 'field of work')]
[('sName', 'Roger Rémondon'), ('pName', 'field of work')]
[('sName', 'Amum-Her-Khepesh-Ef'), ('pName', 'place of burial')]
[('sName', 'Magic in Ancient Egypt'), ('pName', 'facet of')]
[('sName', 'history of ancient Egypt'), ('pName', 'facet of')]
[('sName', 'geography of Ancient Egypt'), ('pName', 'facet of')]
[('sName', 'art of ancient Egypt'), ('pName', 'facet of')]
[('sName', 'Ancient Egyptian philosophy'), ('pName', 'facet of')]
[('sName', 'Tutmania'), ('pName', 'facet of')]
[('sNa

In [174]:
#p P569 date of birth
#p P106 occupation
#p P97 noble title
#obj Q37110 pharaoh

queryString = """
SELECT ?birthdate
WHERE {
    wd:Q44387 wdt:P569 ?dateofbirth .
   ?dateofbirth sc:name ?birthdate.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [9]:

queryString = """
SELECT DISTINCT ?pharaohName ?timeperiod ?timeperiodName
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        wdt:P2348 ?timeperiod ;
        sc:name ?pharaohName .
   ?timeperiod sc:name ?timeperiodName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohName', 'Neferkahor'), ('timeperiod', 'http://www.wikidata.org/entity/Q232211'), ('timeperiodName', 'First Intermediate Period of Egypt')]
[('pharaohName', 'Djehutyemhat'), ('timeperiod', 'http://www.wikidata.org/entity/Q748170'), ('timeperiodName', 'Twenty-third Dynasty of Egypt')]
[('pharaohName', 'Nimlot of Hermopolis'), ('timeperiod', 'http://www.wikidata.org/entity/Q748170'), ('timeperiodName', 'Twenty-third Dynasty of Egypt')]
[('pharaohName', 'Necho II'), ('timeperiod', 'http://www.wikidata.org/entity/Q621917'), ('timeperiodName', 'Late Period of ancient Egypt')]
[('pharaohName', 'Nitocris'), ('timeperiod', 'http://www.wikidata.org/entity/Q177819'), ('timeperiodName', 'Old Kingdom of Egypt')]
[('pharaohName', 'Pepi I Meryre'), ('timeperiod', 'http://www.wikidata.org/entity/Q177819'), ('timeperiodName', 'Old Kingdom of Egypt')]
[('pharaohName', 'Merenre Nemtyemsaf II'), ('timeperiod', 'http://www.wikidata.org/entity/Q177819'), ('timeperiodName', 'Old Kingdom of E

In [12]:
#Q232211 First Intermediate Period of Egypt

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
    ?s ?p wd:Q232211 .
    ?s sc:name ?sName .
    ?p sc:name ?pName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q191324'), ('sName', 'Middle Kingdom of Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('s', 'http://www.wikidata.org/entity/Q177819'), ('sName', 'Old Kingdom of Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q429981'), ('sName', 'Tenth Dynasty of Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('s', 'http://www.wikidata.org/entity/Q64211842'), ('sName', 'Funerary stele of Djefi-015197'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period')]
[('s', 'http://www.wikidata.org/entity/Q551466'), ('sName', 'Neferkahor'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period')]
[('s', 'http://www.wikidata.org/entity/Q447117'), ('sName', 'Old Egyptian'), ('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture')]
[('s', 'http://www.wikidata.org

In [13]:
#Q232211 First Intermediate Period of Egypt

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
    wd:Q232211 ?p  ?obj.
    ?p sc:name ?pName .
    ?obj sc:name ?objName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q177819'), ('objName', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q766876'), ('objName', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q11514315'), ('objName', 'historical period')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q191324'), ('objName', 'Middle Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8949162'), ('objName', 'Category:First Intermediate Period of Egypt')]
5


In [25]:
#Q232211 First Intermediate Period of Egypt followed by
queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
    wd:Q232211 wdt:P156  ?obj.
    ?obj sc:name ?objName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q191324'), ('objName', 'Middle Kingdom of Egypt')]
1


In [17]:
#Q191324 Middle Kingdom of Egypt followed by

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
    wd:Q191324 wdt:P156  ?obj.
    ?obj sc:name ?objName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q206715'), ('objName', 'Second Intermediate Period of Egypt')]
1


In [18]:
#Q206715 Second Intermediate Period of Egypt followed by

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
    wd:Q206715 wdt:P156  ?obj.
    ?obj sc:name ?objName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q180568'), ('objName', 'New Kingdom of Egypt')]
1


In [24]:
#Q180568 New Kingdom of Egypt
#pharaoh during New Kingdom of Egypt and their child

queryString = """
SELECT DISTINCT ?pharaohName ?p ?pName ?childName
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        ?p wd:Q180568 ;
        wdt:P40 ?childName .
    ?p sc:name ?pName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohName', 'Ay'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q40930')]
[('pharaohName', 'Ay'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q273979')]
[('pharaohName', 'Thutmose II'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q157899')]
[('pharaohName', 'Thutmose II'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q242405')]
[('pharaohName', 'Thutmose I'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q158025')]
[('pharaohName', 'Thutmose I'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('childName', 'http://www.wikidata.org/entity/Q129234')]
[('ph

In [10]:
queryString = """
SELECT ?pharaohName ?pName ?oName
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        ?p ?o ;
        wdt:P40 ?child .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
    ?child sc:name ?childName .
    FILTER (datatype(?oName) = xsd:dateTime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [19]:
# wdt:P569 date of birth 
#
queryString = """
SELECT ?pharaohName ?birthdate (COUNT(?childName) AS ?numchild)
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        wdt:P569 ?birthdate ;
        wdt:P40 ?child .
    ?child sc:name ?childName .
    FILTER (datatype(?birthdate) = xsd:dateTime)
}
GROUP BY ?pharaohName ?birthdate
ORDER BY DESC (?numchild)

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohName', 'Ramesses II'), ('birthdate', '-1302-01-01T00:00:00Z'), ('numchild', '18')]
[('pharaohName', 'Khufu'), ('birthdate', '-2700-01-01T00:00:00Z'), ('numchild', '14')]
[('pharaohName', 'Sneferu'), ('birthdate', '-2700-01-01T00:00:00Z'), ('numchild', '13')]
[('pharaohName', 'Amenhotep III'), ('birthdate', '-1403-01-01T00:00:00Z'), ('numchild', '10')]
[('pharaohName', 'Ramesses III'), ('birthdate', '-1216-01-01T00:00:00Z'), ('numchild', '10')]
[('pharaohName', 'Akhenaten'), ('birthdate', '-1400-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Darius I of Persia'), ('birthdate', '-549-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Khafra'), ('birthdate', '-2550-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Ahmose I'), ('birthdate', '-1560-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Seqenenre Tao'), ('birthdate', '-1600-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Ptolemy I Soter'), ('birthdate', '-365-01-01T00:00:00Z'), 

In [20]:
queryString = """
SELECT ?pharaohName ?birthdate (COUNT(?childName) AS ?numchild)
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        wdt:P569 ?birthdate ;
        wdt:P40 ?child .
    ?child sc:name ?childName .
    FILTER (datatype(?birthdate) = xsd:dateTime)
}
GROUP BY ?pharaohName ?birthdate
HAVING (COUNT(?childName) > 8)
ORDER BY DESC (?numchild)

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohName', 'Ramesses II'), ('birthdate', '-1302-01-01T00:00:00Z'), ('numchild', '18')]
[('pharaohName', 'Khufu'), ('birthdate', '-2700-01-01T00:00:00Z'), ('numchild', '14')]
[('pharaohName', 'Sneferu'), ('birthdate', '-2700-01-01T00:00:00Z'), ('numchild', '13')]
[('pharaohName', 'Amenhotep III'), ('birthdate', '-1403-01-01T00:00:00Z'), ('numchild', '10')]
[('pharaohName', 'Ramesses III'), ('birthdate', '-1216-01-01T00:00:00Z'), ('numchild', '10')]
[('pharaohName', 'Akhenaten'), ('birthdate', '-1400-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Darius I of Persia'), ('birthdate', '-549-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Khafra'), ('birthdate', '-2550-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Ahmose I'), ('birthdate', '-1560-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Seqenenre Tao'), ('birthdate', '-1600-01-01T00:00:00Z'), ('numchild', '9')]
[('pharaohName', 'Ptolemy I Soter'), ('birthdate', '-365-01-01T00:00:00Z'), 

Final query for this task

In [23]:
#Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).
# write your final query
queryString = """
SELECT ?pharaoh ?pharaohName (COUNT(?childName) AS ?numchild)
WHERE {
    ?pharaoh wdt:P97 wd:Q37110 ;
        sc:name ?pharaohName ;
        wdt:P569 ?birthdate ;
        wdt:P40 ?child .
    ?child sc:name ?childName .
    FILTER (?birthdate >= "-1500-01-01T00:00:00Z"^^xsd:dateTime && ?birthdate <= "-1000-01-01T00:00:00Z"^^xsd:dateTime)
    
}
GROUP BY ?pharaoh ?pharaohName
HAVING (COUNT(?childName) > 8)
ORDER BY DESC (?numchild)

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q1523'), ('pharaohName', 'Ramesses II'), ('numchild', '18')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1528'), ('pharaohName', 'Ramesses III'), ('numchild', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q42606'), ('pharaohName', 'Amenhotep III'), ('numchild', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q81794'), ('pharaohName', 'Akhenaten'), ('numchild', '9')]
4


## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [38]:
# write your queries
#P31 instance of
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
    ?s wdt:P31 wd:Q28171280 ;
        sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Carthage')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('sName', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q193748'), ('sName', 'history of ancient Israel and Judah')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece')]
[('s', 'http://www.wikidata.org/entity/Q41741'), ('sName', 'Thrace')]
[('s', 'http://www.wikidata.org/entity/Q128904'), ('sName', 'Elam')]
[('s', 'http://www.wikidata.org/entity/Q185068'

In [41]:
# write your queries
#Q35355 Ancient Greece

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q11772 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8744'), ('pName', 'economy of topic'), ('obj', 'http://www.wikidata.org/entity/Q2736823'), ('objName', 'economy of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q841090'), ('objName', 'classics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28171280'), ('objName', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('obj', 'http://www.wikidata.org/entity/Q7798'), ('objName', 'history of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7215882'), ('objName', 'Category:Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q11042'), ('objName',

In [25]:
#Q41741 Thrace

queryString = """
SELECT DISTINCT ?p ?pName #?o ?oName
WHERE {
    wd:Q41741 ?p ?o .
      ?p sc:name ?pName .
      ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by')]
6


In [28]:
#Q4752820 Ancient Crete

queryString = """
SELECT DISTINCT ?p ?pName 
WHERE {
    wd:Q4752820 ?p ?o .
      ?p sc:name ?pName .
      ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category")]
4


In [26]:
# P2348 time period 
#Q269678 ancient Near East

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q269678 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1620908'), ('objName', 'historical region')]
[('p', 'http://www.wikidata.org/prop/direct/P706'), ('pName', 'located in/on physical feature'), ('obj', 'http://www.wikidata.org/entity/Q48214'), ('objName', 'Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q4752891'), ('objName', 'ancient Near Eastern studies')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8251167'), ('objName', 'Category:Ancient Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q100243350'), ('objName', 'medieval Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q61737253'), ('ob

In [42]:
# Q7798 history of ancient Greece

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q7798 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q11772'), ('objName', 'Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17544377'), ('objName', 'history of a country or state')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q6594815'), ('objName', 'Category:History of Greek Antiquity by period')]
3


In [44]:
#Q6594815 Category:History of Greek Antiquity by period

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q6594815 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('obj', 'http://www.wikidata.org/entity/Q7798'), ('objName', 'history of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q59541917'), ('objName', 'Wikimedia topic category')]
2


In [45]:
# Q7112550 outline of ancient Greece

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q7112550 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q26884324'), ('objName', 'Wikimedia outline article')]
1


In [102]:
#Q435608 ancient history
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
    wd:Q435608 ?p ?obj .
      ?p sc:name ?pName .
      ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


In [31]:
# write your queries
#P31 instance of
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
    ?ancientcivil wdt:P31 wd:Q28171280 ;
        ?p ?o ; 
        sc:name ?ancientcivilName .
    ?p sc:name ?pName .
    FILTER REGEX(?pName, 'time*')
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P575'), ('pName', 'time of discovery or invention')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('pName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time')]
4


In [33]:
#P31 instance of
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?ancientcivilName ?starttime ?endtime
WHERE {
    ?ancientcivil wdt:P31 wd:Q28171280 ;
        wdt:P580 ?starttime ;
        wdt:P582 ?endtime ;
        sc:name ?ancientcivilName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilName', 'Minoan civilization'), ('starttime', '-3299-01-01T00:00:00Z'), ('endtime', '-1099-01-01T00:00:00Z')]
[('ancientcivilName', 'Minoan civilization'), ('starttime', '-2000-01-01T00:00:00Z'), ('endtime', '-1099-01-01T00:00:00Z')]
[('ancientcivilName', 'Ancient Egypt'), ('starttime', '-4000-01-01T00:00:00Z'), ('endtime', '0395-01-01T00:00:00Z')]
[('ancientcivilName', 'Ancient Greece'), ('starttime', '-1200-01-01T00:00:00Z'), ('endtime', '0600-01-01T00:00:00Z')]
[('ancientcivilName', 'Indus Valley Civilization'), ('starttime', '-2999-01-01T00:00:00Z'), ('endtime', '-1799-01-01T00:00:00Z')]
[('ancientcivilName', 'Israelite period'), ('starttime', '-1199-01-01T00:00:00Z'), ('endtime', '-585-01-01T00:00:00Z')]
6


In [45]:
#P31 instance of
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?ancientcivilName ?starttime ?endtime ?numyears
WHERE {
    ?ancientcivil wdt:P31 wd:Q28171280 ;
        sc:name ?ancientcivilName .
        
    OPTIONAL{
    ?ancientcivil wdt:P580 ?starttime ;
        wdt:P582 ?endtime .
        BIND(year(?endtime) - year(?starttime) AS ?numyears)
    }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilName', 'Minoan civilization'), ('starttime', '-2000-01-01T00:00:00Z'), ('endtime', '-1099-01-01T00:00:00Z'), ('numyears', '901')]
[('ancientcivilName', 'Minoan civilization'), ('starttime', '-3299-01-01T00:00:00Z'), ('endtime', '-1099-01-01T00:00:00Z'), ('numyears', '2200')]
[('ancientcivilName', 'Ancient Egypt'), ('starttime', '-4000-01-01T00:00:00Z'), ('endtime', '0395-01-01T00:00:00Z'), ('numyears', '4395')]
[('ancientcivilName', 'Ancient Greece'), ('starttime', '-1200-01-01T00:00:00Z'), ('endtime', '0600-01-01T00:00:00Z'), ('numyears', '1800')]
[('ancientcivilName', 'Indus Valley Civilization'), ('starttime', '-2999-01-01T00:00:00Z'), ('endtime', '-1799-01-01T00:00:00Z'), ('numyears', '1200')]
[('ancientcivilName', 'Israelite period'), ('starttime', '-1199-01-01T00:00:00Z'), ('endtime', '-585-01-01T00:00:00Z'), ('numyears', '614')]
[('ancientcivilName', 'Sumer')]
[('ancientcivilName', 'Chinese civilization')]
[('ancientcivilName', 'Gaul')]
[('ancientcivilName

Final query for this task

In [46]:
# write your final query
#P31 instance of
#Q28171280 ancient civilization
#ind all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

queryString = """
SELECT ?ancientcivil ?ancientcivilName ?numyears
WHERE {
    ?ancientcivil wdt:P31 wd:Q28171280 ;
        sc:name ?ancientcivilName .
        
    OPTIONAL{
    ?ancientcivil wdt:P580 ?starttime ;
        wdt:P582 ?endtime .
        BIND(year(?endtime) - year(?starttime) AS ?numyears)
    }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivil', 'http://www.wikidata.org/entity/Q134178'), ('ancientcivilName', 'Minoan civilization'), ('numyears', '901')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q134178'), ('ancientcivilName', 'Minoan civilization'), ('numyears', '2200')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q11768'), ('ancientcivilName', 'Ancient Egypt'), ('numyears', '4395')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q11772'), ('ancientcivilName', 'Ancient Greece'), ('numyears', '1800')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q42534'), ('ancientcivilName', 'Indus Valley Civilization'), ('numyears', '1200')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q1551054'), ('ancientcivilName', 'Israelite period'), ('numyears', '614')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q35355'), ('ancientcivilName', 'Sumer')]
[('ancientcivil', 'http://www.wikidata.org/entity/Q25558247'), ('ancientcivilName', 'Chinese civilization')]
[('ancientcivil', 'http://www.wikida

## Task 4
Identify the BGP for archeologist.

In [50]:
# write your queries

#P373 Commons category
#P186 made from material
#Meidum Pyramid (wd:Q13444093)

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q13444093 wdt:P186 ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q23757'), ('objName', 'limestone')]
1


In [51]:

#P373 Commons category
#Meidum Pyramid (wd:Q13444093)

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q13444093 wdt:P373 ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [52]:
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q13444093 wdt:P31 ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1456099'), ('objName', 'step pyramid')]
1


In [54]:
#Q1456099 step pyramid
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q1456099 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4363262'), ('objName', 'smooth-sided pyramid')]
[('obj', 'http://www.wikidata.org/entity/Q12516'), ('objName', 'pyramid')]
2


In [55]:
#Q12516 pyramid
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q12516 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('obj', 'http://www.wikidata.org/entity/Q3181656'), ('objName', 'The Nuttall Encyclopædia')]
[('obj', 'http://www.wikidata.org/entity/Q39297'), ('objName', 'volume')]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('objName', 'Wikipedia:Vital articles')]
[('obj', 'http://www.wikidata.org/entity/Q867541'), ('objName', 'Encyclopædia Britannica 11th edition')]
[('obj', 'http://www.wikidata.org/entity/Q41176'), ('objName', 'building')]
[('obj', 'http://www.wikidata.org/entity/Q11818429'), ('objName', 'Piramida')]
[('obj', 'http://www.wikidata.org/entity/Q7237020'), ('objName', 'Category:Pyramids')]
[('obj', 'http://www.wikidata.org/entity/Q3358290'), ('objName', 'pyramid')]
[('obj', 'http://www.wikidata.org/entity/Q13361861'), ('objName', 'Template:Infobox pyramid')]
10


In [56]:
#Q41176 building
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q41176 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q35473'), ('objName', 'window')]
[('obj', 'http://www.wikidata.org/entity/P580'), ('objName', 'start time')]
[('obj', 'http://www.wikidata.org/entity/P582'), ('objName', 'end time')]
[('obj', 'http://www.wikidata.org/entity/Q13226383'), ('objName', 'facility')]
[('obj', 'http://www.wikidata.org/entity/Q217164'), ('objName', 'floor')]
[('obj', 'http://www.wikidata.org/entity/Q221706'), ('objName', 'ceiling')]
[('obj', 'http://www.wikidata.org/entity/P631'), ('objName', 'structural engineer')]
[('obj', 'http://www.wikidata.org/entity/P3032'), ('objName', 'adjacent building')]
[('obj', 'http://www.wikidata.org/entity/P466'), ('objName', 'occupant')]
[('obj', 'http://www.wikidata.org/entity/P17'), ('objName', 'country')]
[('obj', 'http://www.wikidata.org/entity/Q101314624'), ('objName', 'Lean Logic: A Dictionary for the Future and How to Survive It')]
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of 

In [57]:
#P84 architect
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:P84 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4387609'), ('objName', 'architectural firm')]
[('obj', 'http://www.wikidata.org/entity/Q43831109'), ('objName', 'Wikidata property related to architecture')]
[('obj', 'http://www.wikidata.org/entity/P287'), ('objName', 'designed by')]
[('obj', 'http://www.wikidata.org/entity/Q42973'), ('objName', 'architect')]
[('obj', 'http://www.wikidata.org/prop/direct/P84'), ('objName', 'architect')]
[('obj', 'http://www.wikidata.org/entity/Q11235'), ('objName', 'World Trade Center')]
[('obj', 'http://www.wikidata.org/entity/P193'), ('objName', 'main building contractor')]
[('obj', 'http://www.wikidata.org/entity/Q18618644'), ('objName', 'Wikidata property for items about works')]
[('obj', 'http://www.wikidata.org/entity/Q21502838'), ('objName', 'conflicts-with constraint')]
[('obj', 'http://www.wikidata.org/entity/Q21503247'), ('objName', 'item-requires-statement constraint')]
[('obj', 'http://www.wikidata.org/entity/Q21503250'), ('objName', 'type 

In [59]:
#Q42973 architect
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q42973 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P417'), ('pName', 'patron saint'), ('obj', 'http://www.wikidata.org/entity/Q43669'), ('objName', 'Thomas the Apostle')]
[('p', 'http://www.wikidata.org/prop/direct/P417'), ('pName', 'patron saint'), ('obj', 'http://www.wikidata.org/entity/Q192816'), ('objName', 'Saint Barbara')]
[('p', 'http://www.wikidata.org/prop/direct/P417'), ('pName', 'patron saint'), ('obj', 'http://www.wikidata.org/entity/Q44265'), ('objName', 'Benedict of Nursia')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q6875304'), ('objName', 'Category:Architects')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1273707

In [60]:
#Q323611 building design
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q323611 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2245405'), ('objName', 'key event')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q309100'), ('objName', 'planning')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q14623005'), ('objName', 'draftsperson')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q42973'), ('objName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q836900'), ('objName', 'construction permit')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q21778977'), ('objName', 'building engineer')]
[('p', 'http://www.wikidata.org/p

In [61]:
#Q21778977 building engineer
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q21778977 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q1090219'), ('objName', 'building manager')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q385378'), ('objName', 'construction')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q81096'), ('objName', 'engineer')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q150737'), ('objName', 'architectural engineering')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q108300140'), ('objName', 'occupation group ac

In [63]:
#Q150737 architectural engineering
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q150737 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q3151019'), ('objName', 'architectural engineer')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q11023'), ('objName', 'engineering')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q12271'), ('objName', 'architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q12271'), ('objName', 'architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q12271'), ('objName', 'architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1924856'), ('objName', 'metal construction')]
[('p', 'http://www.wikidata.org/prop

In [64]:
#P279 subclass of
#Q12271 architecture

queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   ?s wdt:P279 wd:Q12271 .
   ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8204876'), ('sName', 'prehistoric architecture')]
[('s', 'http://www.wikidata.org/entity/Q12079348'), ('sName', 'Architecture of Belarus')]
[('s', 'http://www.wikidata.org/entity/Q30134612'), ('sName', 'water architecture')]
[('s', 'http://www.wikidata.org/entity/Q1856130'), ('sName', 'Mycenaan grave architecture')]
[('s', 'http://www.wikidata.org/entity/Q47848'), ('sName', 'sacred architecture')]
[('s', 'http://www.wikidata.org/entity/Q13724255'), ('sName', 'rock-cut architecture')]
[('s', 'http://www.wikidata.org/entity/Q47844'), ('sName', 'landscape architecture')]
[('s', 'http://www.wikidata.org/entity/Q1501585'), ('sName', 'Iranian architecture')]
[('s', 'http://www.wikidata.org/entity/Q212940'), ('sName', 'Islamic architecture')]
[('s', 'http://www.wikidata.org/entity/Q1477476'), ('sName', 'industrial architecture')]
[('s', 'http://www.wikidata.org/entity/Q957904'), ('sName', 'Chinese architecture')]
[('s', 'http://www.wikidata.org/

In [65]:
#Q12271 architecture

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q12271 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2796470'), ('objName', 'timeline of architecture')]
[('obj', 'http://www.wikidata.org/entity/Q56055944'), ('objName', 'art form')]
[('obj', 'http://www.wikidata.org/entity/Q6204331'), ('objName', 'Category:Architecture')]
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('obj', 'http://www.wikidata.org/entity/Q11862829'), ('objName', 'academic discipline')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('objName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q1970746'), ('objName', 'Explanatory Dictionary of the Living Great Russian Language')]
[('obj', 'http://www.wikidata.org/entity/Q385378'), ('objName', 'construction')]
[('obj', 'http://www.wikidata.org/entity/Q4173137'), ('objName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('obj', 'http://www.wikidata.org/entity/Q42973'), ('objName', 'architect')]
[(

In [71]:
#Q21778977 building engineer
#Q3741966 building science
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   wd:Q21778977 ?p ?obj .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1090219'), ('objName', 'building manager')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q385378'), ('objName', 'construction')]
[('obj', 'http://www.wikidata.org/entity/Q81096'), ('objName', 'engineer')]
[('obj', 'http://www.wikidata.org/entity/Q150737'), ('objName', 'architectural engineering')]
[('obj', 'http://www.wikidata.org/entity/Q108300140'), ('objName', 'occupation group according to ISCO')]
[('obj', 'http://www.wikidata.org/entity/Q323611'), ('objName', 'building design')]
[('obj', 'http://www.wikidata.org/entity/Q811459'), ('objName', 'construction manager')]
[('obj', 'http://www.wikidata.org/entity/Q108289369'), ('objName', 'manufacturing, mining, construction and distribution managers')]
[('obj', 'http://www.wikidata.org/entity/Q3666805'), ('objName', 'Chartered Engineer')]
[('obj', 'http://www.wikidata.org/entity/Q106518281'), ('objName', 'Chartered

In [68]:
#Q21778977 building engineer
#Q3741966 building science
queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
    ?s ?p wd:Q3741966 .
   ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q385378'), ('sName', 'construction')]
[('s', 'http://www.wikidata.org/entity/Q323611'), ('sName', 'building design')]
[('s', 'http://www.wikidata.org/entity/Q2656128'), ('sName', 'seismic analysis')]
[('s', 'http://www.wikidata.org/entity/Q17131184'), ('sName', 'settlement')]
[('s', 'http://www.wikidata.org/entity/Q16751046'), ('sName', 'Stephen Selkowitz')]
[('s', 'http://www.wikidata.org/entity/Q7997511'), ('sName', 'Whole Building Design Guide')]
6


In [77]:
#Q1090219 building manager
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q1090219 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as'), ('obj', 'http://www.wikidata.org/entity/Q84312746'), ('objName', 'janitor')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q21778977'), ('objName', 'building engineer')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as'), ('obj', 'http://www.wikidata.org/entity/Q2664461'), ('objName', 'concierge')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q811459'), ('objName', 'construction manager')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q63930359'), ('objName', 'maintenance worker')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q84313594'), ('objName', 'service pro

In [80]:
#Q323611 building design
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:Q323611 ?p ?obj .
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2245405'), ('objName', 'key event')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q309100'), ('objName', 'planning')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q14623005'), ('objName', 'draftsperson')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q42973'), ('objName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q836900'), ('objName', 'construction permit')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q21778977'), ('objName', 'building engineer')]
[('p', 'http://www.wikidata.org/p

In [81]:
#Q323611 building design
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q323611 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q106425407'), ('sName', 'Lokesh Masania'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q811683'), ('sName', 'proposed building or structure'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('s', 'http://www.wikidata.org/entity/Q3150682'), ('sName', 'thermal mass'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('s', 'http://www.wikidata.org/entity/Q836900'), ('sName', 'construction permit'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('s', 'http://www.wikidata.org/entity/Q1404946'), ('sName', 'Service phases according to HOAI'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q1501477'), ('sName', 'Approval planning'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikid

In [83]:
#Q42973 architect
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q42973 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10973090'), ('sName', 'Template:Infobox architect'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q17650601'), ('sName', 'Czesław Świałkowski'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q19826268'), ('sName', 'Andrzej Urban'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q69384'), ('sName', 'Martin Gropius'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q2628666'), ('sName', 'Elissa Aalto'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q273511'), ('sName', 'Aino Aalto'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wik

In [92]:
# Q19826268 Andrzej Urban
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q19826268 ?p ?o.
   ?o sc:name ?oName .
   ?p sc:name ?pName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q42973'), ('oName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q42973'), ('oName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q4981891'), ('oName', 'Bródno Cemetery')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q809'), ('oName', 'Polish')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pName', 'place of death'), ('o', 'http://www.wikidata.org/entity/Q270'), ('oName', 'Warsaw')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581097'), ('oName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'co

In [85]:
#Q42973 architect
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
    wd:Q42973 ?p ?o.
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('pName', 'patron saint'), ('o', 'http://www.wikidata.org/entity/Q43669'), ('oName', 'Thomas the Apostle')]
[('pName', 'patron saint'), ('o', 'http://www.wikidata.org/entity/Q192816'), ('oName', 'Saint Barbara')]
[('pName', 'patron saint'), ('o', 'http://www.wikidata.org/entity/Q44265'), ('oName', 'Benedict of Nursia')]
[('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6875304'), ('oName', 'Category:Architects')]
[('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q12737077'), ('oName', 'occupation')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oName', 'profession')]
[('pName', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q385378'), ('oName', 'construction')]
[('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q5322166'), ('oNa

In [86]:
#Q12737077 occupation
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
    wd:Q12737077 ?p ?o.
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q4048530'), ('oName', 'Category:Occupations')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q151885'), ('oName', 'concept')]
[('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oName', 'profession')]
[('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q4164871'), ('oName', 'position')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q24017414'), ('oName', 'second-order class')]
[('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1807498'), ('oName', 'social position')]
[('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P106'), ('oName', 'occupation')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3095'), ('oName', 'practiced by')]
[('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P425'), ('oName', 'field of this occupation')]
[('pName', '

In [88]:
#P106 occupation
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
    wd:P106 ?p ?o.
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q83338'), ('oName', 'Robin Williams')]
[('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P101'), ('oName', 'field of work')]
[('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P2650'), ('oName', 'interested in')]
[('pName', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/entity/Q12737077'), ('oName', 'occupation')]
[('pName', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oName', 'profession')]
[('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P39'), ('oName', 'position held')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q70564278'), ('oName', 'Wikidata property used as "depicts" (P180) qualifier on Commons')]
[('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q7327'), ('oName', 'Yuri Gagarin')]
[('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.

In [87]:
#Q4048530 Category:Occupations
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
    wd:Q4048530 ?p ?o.
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pName', "category's main topic"), ('oName', 'occupation')]
[('pName', 'instance of'), ('oName', 'Wikimedia category')]
[('pName', 'different from'), ('oName', 'Category:Professions')]
3


In [79]:
#P279 subclass of
#Q1090219 building manager
queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   ?s wdt:P279 wd:Q1090219 .
   ?s sc:name ?sName .

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q84313124'), ('sName', 'building superintendent')]
[('s', 'http://www.wikidata.org/entity/Q84314029'), ('sName', 'kyrkvaktmästare')]
2


In [70]:
#Q16751046 Stephen Selkowitz
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
    wd:Q16751046 ?p ?o .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1133630'), ('oName', 'Lawrence Berkeley National Laboratory')]
[('o', 'http://www.wikidata.org/entity/Q100'), ('oName', 'Boston')]
[('o', 'http://www.wikidata.org/entity/Q13371'), ('oName', 'Harvard University')]
[('o', 'http://www.wikidata.org/entity/Q4927100'), ('oName', 'Stephen')]
[('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('o', 'http://www.wikidata.org/entity/Q6581097'), ('oName', 'male')]
[('o', 'http://www.wikidata.org/entity/Q81096'), ('oName', 'engineer')]
[('o', 'http://www.wikidata.org/entity/Q1026827'), ('oName', 'California Institute of the Arts')]
[('o', 'http://www.wikidata.org/entity/Q102289'), ('oName', 'Buckminster Fuller')]
[('o', 'http://www.wikidata.org/entity/Q3741966'), ('oName', 'building science')]
10


In [51]:
#Q11768 Ancient Egypt
#Q11772 Ancient Greece
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    #wd:Q11768 ?p ?o .
    wd:Q11772 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8744'), ('pName', 'economy of topic'), ('o', 'http://www.wikidata.org/entity/Q2736823'), ('oName', 'economy of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q841090'), ('oName', 'classics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28171280'), ('oName', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q7798'), ('oName', 'history of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7215882'), ('oName', 'Category:Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11042'), ('oName', 'culture')]
[('p', 'htt

In [52]:
#Q435608 ancient history
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE {

    wd:Q435608 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('oName', 'historian of classical antiquity')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'classical studies')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('oName', 'study of history')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('oName', 'Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('oName', 'Ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'aspect of history')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'specialty')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'field of study')]
[('p', 'http:/

In [60]:
#P2578 studies
queryString = """
SELECT DISTINCT ?s ?sName ?oName
WHERE {

    ?s wdt:P2578 ?o .
    ?s sc:name ?sName .
   ?o sc:name ?oName .
   #FILTER REGEX(?sName , '[Aa]rchae*')
   FILTER REGEX(?oName , '[Aa]rchae*')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q23498'), ('sName', 'archaeology'), ('oName', 'archaeological site')]
[('s', 'http://www.wikidata.org/entity/Q7007095'), ('sName', 'Land of Israel studies'), ('oName', 'archaeology of Israel')]
[('s', 'http://www.wikidata.org/entity/Q259209'), ('sName', 'industrial archaeology'), ('oName', 'industrial archaeology site')]
[('s', 'http://www.wikidata.org/entity/Q93439036'), ('sName', 'Heritage Conservation Studies'), ('oName', 'buildings archaeology')]
4


In [63]:
#Q23498 archaeology
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {

    wd:Q23498 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
   #FILTER REGEX(?sName , '[Aa]rchae*')
   #FILTER REGEX(?oName , '[Aa]rchae*')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1066186'), ('oName', 'study of history')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11862829'), ('oName', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19211082'), ('oName', 'Orthodox Theological Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://ww

Final query for this task

In [64]:
# write your final query
#Identify the BGP for archeologist.
#P3095 practiced by
#Q23498 archaeology
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {

    wd:Q23498 wdt:P3095 ?o .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3621491'), ('oName', 'archaeologist')]
1


## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [96]:
# write your queries
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?ancientcivilizations ?ancientcivilizationsName
WHERE {
    ?ancientcivilizations wdt:P31 wd:Q28171280 ;
        sc:name ?ancientcivilizationsName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q35355'), ('ancientcivilizationsName', 'Sumer')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q25558247'), ('ancientcivilizationsName', 'Chinese civilization')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q38060'), ('ancientcivilizationsName', 'Gaul')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q134178'), ('ancientcivilizationsName', 'Minoan civilization')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q173082'), ('ancientcivilizationsName', 'Dacia')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q2429397'), ('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q3327860'), ('ancientcivilizationsName', 'Celtic kingdom of Ireland')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q193748'), ('ancientcivilizationsName', 'history of ancient Israel and Judah')]
[('ancientcivilizations', 'http://www

In [98]:
#Q25558247 Chinese civilization
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
    wd:Q25558247 ?p ?o .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'instance of'), ('oName', 'ancient civilization')]
[('pName', 'partially coincident with'), ('oName', 'China')]
[('pName', 'different from'), ('oName', 'Chinese culture')]
[('pName', 'history of topic'), ('oName', 'history of China')]
[('pName', 'different from'), ('oName', 'Chinese civilization')]
5


In [101]:
#Q25558247 Chinese civilization
queryString = """
SELECT DISTINCT ?sName ?pName
WHERE {
    ?s ?p wd:Q25558247 .
    ?p sc:name ?pName .
    ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'China'), ('pName', 'partially coincident with')]
[('sName', 'history of China'), ('pName', 'facet of')]
[('sName', 'Cao Wei'), ('pName', 'part of')]
[('sName', 'jinshi'), ('pName', 'culture')]
[('sName', 'Chinese mythology'), ('pName', 'culture')]
[('sName', 'Chinese civilization'), ('pName', 'different from')]
[('sName', 'chain whip'), ('pName', 'culture')]
7


In [65]:
#Q28171280 ancient civilization

queryString = """
SELECT DISTINCT ?ancientcivilizations ?ancientcivilizationsName
WHERE {
    ?ancientcivilizations wdt:P31 wd:Q28171280 ;
        sc:name ?ancientcivilizationsName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q35355'), ('ancientcivilizationsName', 'Sumer')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q25558247'), ('ancientcivilizationsName', 'Chinese civilization')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q38060'), ('ancientcivilizationsName', 'Gaul')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q134178'), ('ancientcivilizationsName', 'Minoan civilization')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q173082'), ('ancientcivilizationsName', 'Dacia')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q2429397'), ('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q3327860'), ('ancientcivilizationsName', 'Celtic kingdom of Ireland')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q193748'), ('ancientcivilizationsName', 'history of ancient Israel and Judah')]
[('ancientcivilizations', 'http://www

In [74]:
#Q3621491 archaeologist
queryString = """
SELECT ?archaeologist ?pName ?oName
WHERE {
    wd:Q3621491 ?p ?o ; #archaeologist 
         sc:name ?archaeologist .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('archaeologist', 'archaeologist'), ('pName', 'has list'), ('oName', 'list of archaeologists')]
[('archaeologist', 'archaeologist'), ('pName', 'subclass of'), ('oName', 'cultural studies scholar')]
[('archaeologist', 'archaeologist'), ('pName', 'subclass of'), ('oName', 'social scientist')]
[('archaeologist', 'archaeologist'), ('pName', 'instance of'), ('oName', 'profession')]
[('archaeologist', 'archaeologist'), ('pName', 'instance of'), ('oName', 'occupation')]
[('archaeologist', 'archaeologist'), ('pName', 'patron saint'), ('oName', 'Helena Augusta')]
[('archaeologist', 'archaeologist'), ('pName', 'patron saint'), ('oName', 'Jerome')]
[('archaeologist', 'archaeologist'), ('pName', 'patron saint'), ('oName', 'Damasus I')]
[('archaeologist', 'archaeologist'), ('pName', 'field of this occupation'), ('oName', 'archaeology')]
[('archaeologist', 'archaeologist'), ('pName', "topic's main category"), ('oName', 'Category:Archaeologists')]
10


In [82]:
#Q3621491 archaeologist
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
    ?s ?p wd:Q3621491 ; #archaeologist 
         sc:name ?archaeologist .
    ?p sc:name ?pName .
     ?s sc:name ?sName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3989'), ('pName', 'members have occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('pName', 'domain of saint or deity')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('pName', 'position held')]
12


In [84]:
#Q3621491 archaeologist
#P101 field of work
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
    ?archaeologist ?p wd:Q3621491 ; #archaeologist 
         sc:name ?archaeologistName .
    ?archaeologist wdt:P101 ?o .
    ?o sc:name ?oName .
     FILTER REGEX(?oName, '[Aa]ncient*' )
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1747689'), ('oName', 'Ancient Rome')]
[('o', 'http://www.wikidata.org/entity/Q269678'), ('oName', 'ancient Near East')]
[('o', 'http://www.wikidata.org/entity/Q7798'), ('oName', 'history of ancient Greece')]
[('o', 'http://www.wikidata.org/entity/Q4246634'), ('oName', 'Ancient Greek culture')]
[('o', 'http://www.wikidata.org/entity/Q11772'), ('oName', 'Ancient Greece')]
[('o', 'http://www.wikidata.org/entity/Q41493'), ('oName', 'ancient history')]
[('o', 'http://www.wikidata.org/entity/Q2429397'), ('oName', 'Ancient Carthage')]
[('o', 'http://www.wikidata.org/entity/Q4752916'), ('oName', 'Ancient Roman pottery')]
[('o', 'http://www.wikidata.org/entity/Q337547'), ('oName', 'ancient Roman religion')]
[('o', 'http://www.wikidata.org/entity/Q435608'), ('oName', 'ancient history')]
[('o', 'http://www.wikidata.org/entity/Q35966'), ('oName', 'Ancient Celts')]
[('o', 'http://www.wikidata.org/entity/Q738680'), ('oName', 'pottery of ancient Greece'

In [78]:
#Q8463726 Chengzuo Shang
queryString = """
SELECT ?p ?pName ?o ?oName 
WHERE {
    wd:Q8463726 ?p ?o .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q10923207'), ('oName', 'Yanliu Shang')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q32751'), ('oName', 'National Sun Yat-sen University')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581097'), ('oName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q16572'), ('oName', 'Guangzhou')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q16952'), ('oName', 'Peking University')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')

In [80]:
queryString = """
SELECT ?p ?pName ?o ?oName 
WHERE {
    ?archaeologist wdt:P106 wd:Q3621491 ; #archaeologist  
             ?p ?o .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q10923207'), ('oName', 'Yanliu Shang')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q18397302'), ('oName', 'Laach')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pName', 'relative'), ('o', 'http://www.wikidata.org/entity/Q3839667'), ('oName', 'Luigi Gaetano Marini')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q15876247'), ('oName', 'Henry Balfour')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q36871867'), ('oName', 'Stubbings')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q106264490'), ('oName', 'Rebuffat')]
[('p', 'http://www.wikidata.org/prop/direct/P185'), ('pName', '

In [81]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE {
    ?archaeologist wdt:P106 wd:Q3621491 . #archaeologist  
    ?s ?p ?archaeologist .
    ?p sc:name ?pName .
     ?s sc:name ?sName .
     
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10923207'), ('sName', 'Yanliu Shang'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child')]
[('s', 'http://www.wikidata.org/entity/Q15274985'), ('sName', "Dominique-Vivant Denon: l'oeil de Napoléon"), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q15729291'), ('sName', 'Marino Marini'), ('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pName', 'relative')]
[('s', 'http://www.wikidata.org/entity/Q15733244'), ('sName', 'Daring Do'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q15876247'), ('sName', 'Henry Balfour'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q15892068'), ('sName', 'Ἀγχίσου λιμήν (Pauly-Wissowa)'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('pName', 'author')]
[('s', 'http://www.wikid

In [87]:
#Q3621491 archaeologist
#P101 field of work
queryString = """
SELECT ?ancientcivilizationsName
WHERE {
    ?archaeologist ?p wd:Q3621491 ; #archaeologist 
         sc:name ?archaeologistName .
    ?archaeologist wdt:P101 ?ancientcivilizations . #archaeologist name
    ?ancientcivilizations wdt:P31 wd:Q28171280 ; #instance of ancient civilization
        sc:name ?ancientcivilizationsName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilizationsName', 'Gaul')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Gaul')]
[('ancientcivilizationsName', 'Gaul')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Mesopotamia')]
[('ancientcivilizationsName', 'Ancient Carthage')]
[('ancientcivilizationsName', 'Israelite period')]
[('ancientcivilizationsName', 'history of ancient Israel and Judah')]
[('ancientcivilizationsName', 'Ancient Greece')]
[('ancientcivilizationsName', 'Thrace')]
[('ancientciviliza

In [92]:
#Q3621491 archaeologist
#P106 occupation
#P31 instance of #Q28171280 ancient civilization
queryString = """
SELECT DISTINCT ?p1Name ?p2Name ?ancientcivilizationsName
WHERE {
    ?archaeologist ?p1 wd:Q3621491 ; #archaeologist 
         sc:name ?archaeologistName .
    ?archaeologist ?p2 ?ancientcivilizations . #archaeologist name
    ?ancientcivilizations wdt:P31 wd:Q28171280 ; #instance of ancient civilization
        sc:name ?ancientcivilizationsName .
    ?p1 sc:name ?p1Name .
    ?p2 sc:name ?p2Name .

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Gaul')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Ancient Greece')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Ancient Carthage')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Mesopotamia')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Israelite period')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'history of ancient Israel and Judah')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Thrace')]
[('p1Name', 'occupation'), ('p2Name', 'field of work'), ('ancientcivilizationsName', 'Maya civilization')]
8


Final query for this task

In [101]:
# write your final query
#For each ancient civilization find the number of archeologists who worked on it
#Q3621491 archaeologist
#P106 occupation
#P31 instance of #Q28171280 ancient civilization
queryString = """
SELECT ?ancientcivilizations ?ancientcivilizationsName  (COUNT(?archaeologistName) AS ?numarchaeologist)
WHERE {

    ?ancientcivilizations wdt:P31 wd:Q28171280 ; #instance of ancient civilization
        sc:name ?ancientcivilizationsName .
    OPTIONAL
    {
    ?archaeologist wdt:P106 wd:Q3621491 ;
         sc:name ?archaeologistName . #archaeologist name
    ?archaeologist wdt:P101 ?ancientcivilizations . #archaeologist works on ancient civilization
    }

}
GROUP BY ?ancientcivilizations ?ancientcivilizationsName 
ORDER BY DESC (?numarchaeologist)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q11772'), ('ancientcivilizationsName', 'Ancient Greece'), ('numarchaeologist', '7')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q2429397'), ('ancientcivilizationsName', 'Ancient Carthage'), ('numarchaeologist', '7')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q38060'), ('ancientcivilizationsName', 'Gaul'), ('numarchaeologist', '3')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q1551054'), ('ancientcivilizationsName', 'Israelite period'), ('numarchaeologist', '2')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q11767'), ('ancientcivilizationsName', 'Mesopotamia'), ('numarchaeologist', '1')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q193748'), ('ancientcivilizationsName', 'history of ancient Israel and Judah'), ('numarchaeologist', '1')]
[('ancientcivilizations', 'http://www.wikidata.org/entity/Q41741'), ('ancientcivilizationsName', 'Thrace'), ('numarch

## Task 6
Find the BGP for deity.

In [103]:
# write your queries
# wd:Q41127 Poseidon
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE {
    wd:Q41127 ?p ?o .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
     FILTER (REGEX(?pName , "[Dd]eity*") || REGEX(?oName, "[Dd]eity*"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1916821'), ('oName', 'water deity')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q22989102'), ('oName', 'Greek deity')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('pName', 'domain of saint or deity'), ('o', 'http://www.wikidata.org/entity/Q165'), ('oName', 'sea')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('pName', 'domain of saint or deity'), ('o', 'http://www.wikidata.org/entity/Q7944'), ('oName', 'earthquake')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('pName', 'domain of saint or deity'), ('o', 'http://www.wikidata.org/entity/Q1265288'), ('oName', 'horse breeding')]
5


In [104]:
#Q1916821 water deity
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE {
    wd:Q1916821 ?p ?o .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13405593'), ('oName', 'nature deity')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('pName', 'domain of saint or deity'), ('o', 'http://www.wikidata.org/entity/Q283'), ('oName', 'water')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1936706'), ('oName', 'water sprite')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8163944'), ('oName', 'Category:Water deities')]
4


In [105]:
#P2925 domain of saint or deity
queryString = """
SELECT ?s ?sName ?o ?oName
WHERE {
    ?s wdt:P2925 ?o .
    ?s sc:name ?sName .
     ?o sc:name ?oName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2736992'), ('sName', 'Sumayyah bint Khayyat'), ('o', 'http://www.wikidata.org/entity/Q643638'), ('oName', 'Shahid')]
[('s', 'http://www.wikidata.org/entity/Q3074497'), ('sName', 'Fluonia'), ('o', 'http://www.wikidata.org/entity/Q12171'), ('oName', 'menstruation')]
[('s', 'http://www.wikidata.org/entity/Q444050'), ('sName', 'Bartolomea Capitanio'), ('o', 'http://www.wikidata.org/entity/Q2366933'), ('oName', 'Sisters of Charity of Saints Bartolomea Capitanio and Vincenza Gerosa (SCCG)')]
[('s', 'http://www.wikidata.org/entity/Q5791704'), ('sName', 'Christ of the Good Death'), ('o', 'http://www.wikidata.org/entity/Q93470005'), ('oName', 'spanish legionary')]
[('s', 'http://www.wikidata.org/entity/Q444050'), ('sName', 'Bartolomea Capitanio'), ('o', 'http://www.wikidata.org/entity/Q101202'), ('oName', 'Lovere')]
[('s', 'http://www.wikidata.org/entity/Q452435'), ('sName', 'Bona of Pisa'), ('o', 'http://www.wikidata.org/entity/Q101539'), ('oName

In [106]:
# Q13405593 nature deity
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE {
    wd:Q13405593 ?p ?o .
    ?p sc:name ?pName .
     ?o sc:name ?oName .
     
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1916821'), ('oName', 'water deity')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q20828805'), ('oName', 'sprite')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q178885'), ('oName', 'deity')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q739737'), ('oName', 'Cernunnos')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6630292'), ('oName', 'list of nature deities')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7721989'), ('oName', 'Category:Nature deities')]
[('p', 'http://www.wikidata.org/prop/direct

Final query for this task

In [108]:
# write your final query
# Q13405593 nature deity
#P279 subclass of
queryString = """
SELECT ?deity ?deityName
WHERE {
    wd:Q13405593 wdt:P279 ?deity .
     ?deity sc:name ?deityName .
     FILTER REGEX(?deityName , 'deity')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q178885'), ('deityName', 'deity')]
1


## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [109]:
# write your queries
# Q22989102 Greek deity
queryString = """
SELECT ?p ?pName ?s ?sName 
WHERE {
    wd:Q22989102 ?p ?s .
     ?p sc:name ?pName .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('s', 'http://www.wikidata.org/entity/Q95074'), ('sName', 'fictional character')]
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture'), ('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('s', 'http://www.wikidata.org/entity/Q22988604'), ('sName', 'mythological Greek character')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('s', 'http://www.wikidata.org/entity/Q178885'), ('sName', 'deity')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q34726'), ('sName', 'Greek mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q855270'), ('sName', 'religion in ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [112]:
# Q34726 Greek mythology
queryString = """
SELECT ?p ?pName ?o ?oName 
WHERE {
    wd:Q34726 ?p ?o .
     ?p sc:name ?pName .
     ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q1367434'), ('oName', 'classical mythology')]
[('p', 'http://www.wikidat

In [111]:
# Q22988604 mythological Greek character
queryString = """
SELECT ?p ?pName ?s ?sName 
WHERE {
    wd:Q22988604 ?p ?s .
     ?p sc:name ?pName .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('s', 'http://www.wikidata.org/entity/Q95074'), ('sName', 'fictional character')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q34726'), ('sName', 'Greek mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('s', 'http://www.wikidata.org/entity/Q4271324'), ('sName', 'mythical character')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('s', 'http://www.wikidata.org/entity/Q8500207'), ('sName', 'Category:Characters in Greek mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('s', 'http://www.wikidata.org/entity/Q20059824'), ('sName', 'list of mythological Greek characters')]
5


In [117]:
# Q22989102 Greek deity
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {
    ?s ?p wd:Q22989102 .
     ?p sc:name ?pName .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5081127'), ('sName', 'Acmon'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q366533'), ('sName', 'Adranus'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q3362052'), ('sName', 'Pandora'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q6612'), ('sName', 'Charon'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q2144233'), ('sName', 'Pheme'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1120723'), ('sName', 'Pheme'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1862189'), ('sName', 'Paean'), ('p', 'http://www.wikidata.org/prop/di

In [129]:
# Q5081127 Acmon
queryString = """
SELECT DISTINCT ?greekdeity ?greekdeityName ?p2 ?p2Name ?o2?o2Name
WHERE {
    ?greekdeity ?p1 wd:Q22989102 ;
         sc:name ?greekdeityName ;
         ?p2 ?o2.
     ?p1 sc:name ?p1Name .
     ?p2 sc:name ?p2Name .
     ?o2 sc:name ?o2Name .
     
     FILTER REGEX(?o2Name , "[Oo]lym*")
     
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('greekdeity', 'http://www.wikidata.org/entity/Q37122'), ('greekdeityName', 'Athena'), ('p2', 'http://www.wikidata.org/prop/direct/P1299'), ('p2Name', 'depicted by'), ('o2', 'http://www.wikidata.org/entity/Q107043631'), ('o2Name', 'Athena statue at Olympia')]
[('greekdeity', 'http://www.wikidata.org/entity/Q37340'), ('greekdeityName', 'Apollo'), ('p2', 'http://www.wikidata.org/prop/direct/P1299'), ('p2Name', 'depicted by'), ('o2', 'http://www.wikidata.org/entity/Q107043625'), ('o2Name', 'Apollo statue in Sicyonian treasury, Olympia')]
[('greekdeity', 'http://www.wikidata.org/entity/Q34201'), ('greekdeityName', 'Zeus'), ('p2', 'http://www.wikidata.org/prop/direct/P1299'), ('p2Name', 'depicted by'), ('o2', 'http://www.wikidata.org/entity/Q107043626'), ('o2Name', 'Zeus statue in Sicyonian treasury, Olympia')]
[('greekdeity', 'http://www.wikidata.org/entity/Q38012'), ('greekdeityName', 'Hera'), ('p2', 'http://www.wikidata.org/prop/direct/P361'), ('p2Name', 'part of'), ('o2', 'http

In [130]:
#Q101609 Twelve Olympians
#P361 part of
queryString = """
SELECT DISTINCT ?twelveolympiansName
WHERE {
    ?twelveolympians wdt:P361 wd:Q101609 .
     ?twelveolympians sc:name ?twelveolympiansName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('twelveolympiansName', 'Apollo')]
[('twelveolympiansName', 'Athena')]
[('twelveolympiansName', 'Hera')]
[('twelveolympiansName', 'Artemis')]
[('twelveolympiansName', 'Hephaestus')]
[('twelveolympiansName', 'Hermes')]
[('twelveolympiansName', 'Poseidon')]
[('twelveolympiansName', 'Hestia')]
[('twelveolympiansName', 'Zeus')]
[('twelveolympiansName', 'Aphrodite')]
[('twelveolympiansName', 'Demeter')]
[('twelveolympiansName', 'Ares')]
12


Final query for this task

In [131]:
# write your final query
#Q101609 Twelve Olympians
#P361 part of
queryString = """
SELECT DISTINCT ?twelveolympians ?twelveolympiansName
WHERE {
    ?twelveolympians wdt:P361 wd:Q101609 .
     ?twelveolympians sc:name ?twelveolympiansName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('twelveolympians', 'http://www.wikidata.org/entity/Q37340'), ('twelveolympiansName', 'Apollo')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q37122'), ('twelveolympiansName', 'Athena')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q38012'), ('twelveolympiansName', 'Hera')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q39503'), ('twelveolympiansName', 'Artemis')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q44384'), ('twelveolympiansName', 'Hephaestus')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q41484'), ('twelveolympiansName', 'Hermes')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q41127'), ('twelveolympiansName', 'Poseidon')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q41419'), ('twelveolympiansName', 'Hestia')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q34201'), ('twelveolympiansName', 'Zeus')]
[('twelveolympians', 'http://www.wikidata.org/entity/Q35500'), ('twelveolympiansName', 'Aphrodite')]
[('twe

## Task 8
Compare ancient Egypt with ancient Egypt and find how many deities each one has.

In [132]:
# write your queries
#Q22989102 Greek deity
#Q11772 Ancient Greece
#Q11768 Ancient Egypt
queryString = """
SELECT DISTINCT ?pName ?sName
WHERE {
    wd:Q11772 ?p ?s .
     ?p sc:name ?pName .
     ?s sc:name ?sName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'economy of topic'), ('sName', 'economy of ancient Greece')]
[('pName', 'studied by'), ('sName', 'classics')]
[('pName', 'instance of'), ('sName', 'ancient civilization')]
[('pName', 'history of topic'), ('sName', 'history of ancient Greece')]
[('pName', "topic's main category"), ('sName', 'Category:Ancient Greece')]
[('pName', 'instance of'), ('sName', 'culture')]
[('pName', 'instance of'), ('sName', 'style')]
[('pName', 'described by source'), ('sName', 'The Nuttall Encyclopædia')]
[('pName', 'instance of'), ('sName', 'cultural region')]
[('pName', 'indigenous to'), ('sName', 'Greece')]
[('pName', 'continent'), ('sName', 'Europe')]
[('pName', 'continent'), ('sName', 'Asia')]
[('pName', 'on focus list of Wikimedia project'), ('sName', 'Wikipedia:Vital articles')]
[('pName', 'shares border with'), ('sName', 'Persian Empire')]
[('pName', 'instance of'), ('sName', 'historical region')]
[('pName', 'shares border with'), ('sName', 'Illyria')]
[('pName', 'shares border wi

In [134]:
# Q22989102 Greek deity
queryString = """
SELECT DISTINCT ?s ?sName 
WHERE {
    ?s wdt:P31 wd:Q22989102 .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5081127'), ('sName', 'Acmon')]
[('s', 'http://www.wikidata.org/entity/Q366533'), ('sName', 'Adranus')]
[('s', 'http://www.wikidata.org/entity/Q3362052'), ('sName', 'Pandora')]
[('s', 'http://www.wikidata.org/entity/Q6612'), ('sName', 'Charon')]
[('s', 'http://www.wikidata.org/entity/Q2144233'), ('sName', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1120723'), ('sName', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1862189'), ('sName', 'Paean')]
[('s', 'http://www.wikidata.org/entity/Q1775444'), ('sName', 'Phosphorus')]
[('s', 'http://www.wikidata.org/entity/Q178417'), ('sName', 'Eos')]
[('s', 'http://www.wikidata.org/entity/Q59697'), ('sName', 'Hesperus')]
[('s', 'http://www.wikidata.org/entity/Q1084868'), ('sName', 'Lyssa')]
[('s', 'http://www.wikidata.org/entity/Q1294659'), ('sName', 'Telesphorus')]
[('s', 'http://www.wikidata.org/entity/Q18423351'), ('sName', 'Basileia')]
[('s', 'http://www.wikidata.org/entity/Q185670'), ('sNa

In [138]:

queryString = """
SELECT DISTINCT ?s ?sName ?o ?oName 
WHERE {
    ?s wdt:P31 ?o .
     ?s sc:name ?sName .
     ?o sc:name ?oName .
     FILTER REGEX(?oName , '[Ee]gypt*')
     FILTER REGEX(?oName , 'deity*')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145915'), ('sName', 'Nekhbet'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q58373'), ('sName', 'Amun'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q2620491'), ('sName', 'Werethekau'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q170569'), ('sName', 'Atum'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q211611'), ('sName', 'Anuket'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q146313'), ('sName', 'Sobek'), ('o', 'http://www.wikidata.org/entity/Q146083'), ('oName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q1252904'),

In [139]:
#Q146083 Ancient Egyptian deity
queryString = """
SELECT DISTINCT ?s ?sName 
WHERE {
    ?s wdt:P31 wd:Q146083 .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145915'), ('sName', 'Nekhbet')]
[('s', 'http://www.wikidata.org/entity/Q58373'), ('sName', 'Amun')]
[('s', 'http://www.wikidata.org/entity/Q2620491'), ('sName', 'Werethekau')]
[('s', 'http://www.wikidata.org/entity/Q170569'), ('sName', 'Atum')]
[('s', 'http://www.wikidata.org/entity/Q211611'), ('sName', 'Anuket')]
[('s', 'http://www.wikidata.org/entity/Q146313'), ('sName', 'Sobek')]
[('s', 'http://www.wikidata.org/entity/Q1252904'), ('sName', 'Ra')]
[('s', 'http://www.wikidata.org/entity/Q130227'), ('sName', 'Aten')]
[('s', 'http://www.wikidata.org/entity/Q133343'), ('sName', 'Hathor')]
[('s', 'http://www.wikidata.org/entity/Q845614'), ('sName', 'Maahes')]
[('s', 'http://www.wikidata.org/entity/Q116273'), ('sName', 'Tefnut')]
[('s', 'http://www.wikidata.org/entity/Q1936121'), ('sName', 'Min-Kamutef')]
[('s', 'http://www.wikidata.org/entity/Q1627813'), ('sName', 'Horus the Child')]
[('s', 'http://www.wikidata.org/entity/Q790000'), ('sName'

In [142]:
#Q146083 Ancient Egyptian deity
queryString = """
SELECT DISTINCT COUNT((?sName) AS ?num)
WHERE {
    ?s wdt:P31 wd:Q146083 .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', '156')]
1


In [143]:
# Q22989102 Greek deity
queryString = """
SELECT DISTINCT COUNT((?sName) AS ?num)
WHERE {
    ?s wdt:P31 wd:Q22989102 .
     ?s sc:name ?sName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', '241')]
1


Final query for this task

In [149]:
# write your final query
#Q146083 Ancient Egyptian deity
# Q22989102 Greek deity
queryString = """
SELECT ?egyptianName COUNT(DISTINCT(?egyptiandeity) AS ?numegyptiandeity) ?greekName COUNT(DISTINCT(?greekdeity) AS ?numgreekdeity)
WHERE {
    wd:Q146083 sc:name ?egyptianName .
    wd:Q22989102 sc:name ?greekName .
    
    ?egyptiandeity wdt:P31 wd:Q146083 ;
       sc:name ?egyptiandeityName .

    ?greekdeity wdt:P31 wd:Q22989102 ;
           sc:name ?greekdeityName .

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('egyptianName', 'Ancient Egyptian deity'), ('callret-1', '156'), ('greekName', 'Greek deity'), ('callret-3', '241')]
1
